# EDA con Enfoque en CAC y LTV

Este cuaderno se centra en **preguntas de negocio** que generan valor al entender la relación entre:
- **CAC**: Costo de Adquisición de Cliente.
- **LTV**: Valor de Vida del Cliente.

Adicionalmente, explora aspectos clave como **suscripción**, **descuentos**, **categoría** del producto, **frecuencia de compras**, **método de pago**, etc. Con estas preguntas podrás tomar **decisiones estratégicas** sobre tu modelo de negocio, campañas de marketing y retención de clientes.

## Contenido:
1. **Importar librerías y cargar datos**
2. **Inspección inicial** (dimensiones, tipos de datos, nulos)
3. **Limpieza básica** (si es necesaria)
4. **Creación de columna** ratio `LTV/CAC` (rentabilidad por cliente)
5. **Preguntas de negocio** (groupby, filtros, etc.)
6. **Conclusiones**

In [1]:
!wget !wget https://github.com/javierherrera1996/IntroMarketingAnalytics/raw/refs/heads/main/PrimerCorte/shopping_trends_LTV_CAC.csv

--2025-03-06 01:55:06--  http://!wget/
Resolving !wget (!wget)... failed: Name or service not known.
wget: unable to resolve host address ‘!wget’
--2025-03-06 01:55:06--  https://github.com/javierherrera1996/IntroMarketingAnalytics/raw/refs/heads/main/PrimerCorte/shopping_trends_LTV_CAC.csv
Resolving github.com (github.com)... 140.82.113.3
Connecting to github.com (github.com)|140.82.113.3|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/javierherrera1996/IntroMarketingAnalytics/refs/heads/main/PrimerCorte/shopping_trends_LTV_CAC.csv [following]
--2025-03-06 01:55:06--  https://raw.githubusercontent.com/javierherrera1996/IntroMarketingAnalytics/refs/heads/main/PrimerCorte/shopping_trends_LTV_CAC.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.111.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HT

## 1. Importar librerías y cargar datos
Asegúrate de tener tu archivo CSV con las columnas mencionadas: `CAC`, `LTV`, `Purchase Amount (USD)`, etc. Ajusta la ruta según tu archivo real.

In [2]:
import pandas as pd
import numpy as np


# Cargar el dataset
df = pd.read_csv("shopping_trends_LTV_CAC.csv")


# Mostramos las primeras filas para verificar
df.head()

,Customer ID,Age,Gender,Item Purchased,Category,Purchase Amount (USD),Location,Size,Color,Season,...,Shipping Type,Discount Applied,Promo Code Used,Previous Purchases,Preferred Payment Method,Frequency of Purchases,CAC,LTV,Rentabilidad,LTV_CAC_Ratio
0,1,55,Male,Blouse,Clothing,53,Kentucky,L,Gray,Winter,...,Express,Yes,Yes,14,Venmo,Fortnightly,112.0,1512,16.434783,13.500000
1,2,19,Male,Sweater,Clothing,64,Maine,L,Maroon,Winter,...,Express,Yes,Yes,2,Cash,Fortnightly,95.0,1497,19.960000,15.757895
2,3,50,Male,Jeans,Clothing,73,Massachusetts,S,Maroon,Spring,...,Free Shipping,Yes,Yes,23,Credit Card,Weekly,32.0,1137,94.750000,35.531250
3,4,21,Male,Sandals,Footwear,90,Rhode Island,M,Maroon,Spring,...,Next Day Air,Yes,Yes,49,PayPal,Weekly,NaN,1914,26.957746,26.957746
4,5,45,Male,Blouse,Clothing,49,Oregon,M,Turquoise,Spring,...,Free Shipping,Yes,Yes,31,PayPal,Annually,47.0,1300,48.148148,27.659574


## 2. Inspección inicial
1. **Dimensiones (filas x columnas)**
2. **Info general**: tipos de dato, nulos, etc.
3. **Valores nulos** por columna.

Si existen valores nulos en columnas importantes (por ejemplo, `CAC`, `LTV`, `Customer ID`), evalúa si los eliminarás o imputarás.

In [3]:
df.isnull().sum()

,0
Customer ID,0
Age,0
Gender,0
Item Purchased,0
Category,0
Purchase Amount (USD),0
Location,0
Size,0
Color,0
Season,0


## 3. Limpieza básica
Por ejemplo, si detectas `CAC <= 0` o `LTV <= 0` y consideras que no tienen sentido en tu negocio, podrías filtrar esos registros. Ajusta según tu criterio.

In [4]:
df[df["LTV"]<0]

,Customer ID,Age,Gender,Item Purchased,Category,Purchase Amount (USD),Location,Size,Color,Season,...,Shipping Type,Discount Applied,Promo Code Used,Previous Purchases,Preferred Payment Method,Frequency of Purchases,CAC,LTV,Rentabilidad,LTV_CAC_Ratio


In [5]:
df[df["CAC"] < 0]

,Customer ID,Age,Gender,Item Purchased,Category,Purchase Amount (USD),Location,Size,Color,Season,...,Shipping Type,Discount Applied,Promo Code Used,Previous Purchases,Preferred Payment Method,Frequency of Purchases,CAC,LTV,Rentabilidad,LTV_CAC_Ratio


## 4. Creación de columna: Ratio LTV/CAC
Es una métrica clave de la **rentabilidad** del cliente. Un valor > 1 indica que, en teoría, el cliente retorna más de lo que cuesta adquirirlo.

In [6]:
df["Ratio LTV/CAC"] = df["LTV"] / df["CAC"]
df.head()

,Customer ID,Age,Gender,Item Purchased,Category,Purchase Amount (USD),Location,Size,Color,Season,...,Discount Applied,Promo Code Used,Previous Purchases,Preferred Payment Method,Frequency of Purchases,CAC,LTV,Rentabilidad,LTV_CAC_Ratio,Ratio LTV/CAC
0,1,55,Male,Blouse,Clothing,53,Kentucky,L,Gray,Winter,...,Yes,Yes,14,Venmo,Fortnightly,112.0,1512,16.434783,13.500000,13.500000
1,2,19,Male,Sweater,Clothing,64,Maine,L,Maroon,Winter,...,Yes,Yes,2,Cash,Fortnightly,95.0,1497,19.960000,15.757895,15.757895
2,3,50,Male,Jeans,Clothing,73,Massachusetts,S,Maroon,Spring,...,Yes,Yes,23,Credit Card,Weekly,32.0,1137,94.750000,35.531250,35.531250
3,4,21,Male,Sandals,Footwear,90,Rhode Island,M,Maroon,Spring,...,Yes,Yes,49,PayPal,Weekly,NaN,1914,26.957746,26.957746,NaN
4,5,45,Male,Blouse,Clothing,49,Oregon,M,Turquoise,Spring,...,Yes,Yes,31,PayPal,Annually,47.0,1300,48.148148,27.659574,27.659574


## 5. Preguntas de Negocio
En vez de usar gráficas o correlaciones, nos centramos en **preguntas que generen valor** al análisis. Cada una se resuelve con **groupby, filtros o estadísticas** simples.

1. **¿Cuántos clientes únicos hay y cuál es la edad promedio?**
2. **¿Cuál es el CAC promedio y el LTV promedio en todo el dataset?**
3. **¿Cuál es la ratio LTV/CAC promedio?**
4. **¿Qué categoría tiene el mayor LTV en promedio?**
5. **¿En cuál categoría se justifica más el CAC?** (categoría con mayor ratio LTV/CAC)
6. **¿Los clientes suscritos tienen LTV mayor que los no suscritos?**
7. **¿Cuál es el método de pago más frecuente y su LTV promedio?**
8. **¿Quiénes usan descuentos con mayor frecuencia?** (Por ejemplo, ¿cierto género, cierto rango de edad?)
9. **¿Cuántos clientes aprovechan códigos promocionales (Promo Code Used)?** y su ratio LTV/CAC
10. **¿Cuál es la frecuencia de compra más común (Frequency of Purchases)?** y su LTV asociado.
11. **¿Cuál es la ubicación (Location) con mayor ratio LTV/CAC?**
12. **Entre los clientes con `Previous Purchases` > X, cuál es el gasto promedio y LTV promedio?**


### 5.1 ¿Cuántos clientes únicos hay y cuál es la edad promedio?

In [7]:
Totalclients = df["Customer ID"].nunique()
Averageage = df["Age"].mean()

print(f"Total clients: {Totalclients}")
print(f"Average age: {Averageage}")

Total clients: 3900
Average age: 44.06846153846154


### 5.2 ¿Cuál es el CAC promedio y el LTV promedio en todo el dataset?

In [8]:
df.groupby("Customer ID").agg({"CAC":"mean","LTV":"mean"})

,CAC,LTV
Customer ID,,
1,112.0,1512.0
2,95.0,1497.0
3,32.0,1137.0
4,NaN,1914.0
5,47.0,1300.0
...,...,...
3896,77.0,674.0
3897,44.0,1292.0
3898,97.0,1378.0


### 5.3 ¿Cuál es la ratio LTV/CAC promedio?

In [9]:
df["Ratio LTV/CAC"].mean()

22.446982949947316

### 5.4 ¿Qué categoría tiene el mayor LTV en promedio?

In [10]:
df.groupby("Category").agg({"LTV":"mean"}).sort_values(by="LTV",ascending=False)

,LTV
Category,
Accessories,1145.992742
Footwear,1099.308848
Clothing,1095.688543
Outerwear,1056.040123


### 5.5 ¿En cuál categoría se justifica más el CAC? (mayor ratio LTV/CAC)

In [11]:
df.groupby("Category").agg({"Ratio LTV/CAC":"mean"}).sort_values(by="Ratio LTV/CAC",ascending=False)

,Ratio LTV/CAC
Category,
Accessories,29.281313
Outerwear,26.319371
Clothing,16.845818
Footwear,NaN


### 5.6 ¿Los clientes suscritos tienen LTV mayor que los no suscritos?

In [12]:
df.groupby("Subscription Status").agg({"LTV":"mean"})

,LTV
Subscription Status,
No,1113.425009
Yes,1096.831909


### 5.7 ¿Cuál es el método de pago más frecuente y su LTV promedio?

In [13]:
df.groupby("Payment Method").agg({"LTV":"mean"}).sort_values(by="LTV",ascending=False)

,LTV
Payment Method,
Cash,1151.208333
Bank Transfer,1139.090190
Credit Card,1097.510057
Venmo,1090.260337
Debit Card,1089.439179
PayPal,1087.108150


### 5.8 ¿Quiénes usan descuentos con mayor frecuencia?
Podemos ver si hay algún grupo (por género, rango de edad, etc.) que destaca. A modo de ejemplo, segmentaremos por género:

In [23]:
df.groupby('Gender')['Discount Applied'].value_counts()

Gender  Discount Applied
Female  No                  1248
Male    Yes                 1677
        No                   975
Name: count, dtype: int64

Podrías repetir el ejercicio segmentando por **Age** (definiendo rangos) o por **Location**.

### 5.9 ¿Cuántos clientes aprovechan códigos promocionales y cuál es su ratio LTV/CAC?
El uso de promos puede modificar la rentabilidad.

In [14]:
df.groupby("Promo Code Used").agg({"Ratio LTV/CAC":"mean"})

,Ratio LTV/CAC
Promo Code Used,
No,22.252655
Yes,22.705036


### 5.10 ¿Cuál es la frecuencia de compra más común (Frequency of Purchases) y su LTV?
Esto ayuda a ver si, por ejemplo, los clientes que compran semanalmente tienen un LTV más alto.

In [15]:
df.groupby("Frequency of Purchases").agg({"LTV":"mean"})

,LTV
Frequency of Purchases,
Annually,1107.472028
Bi-Weekly,1182.279707
Every 3 Months,1072.965753
Fortnightly,1106.501845
Monthly,1087.001808
Quarterly,1098.229130
Weekly,1111.230056


### 5.11 ¿Cuál es la ubicación (Location) con mayor ratio LTV/CAC?

In [16]:
df.groupby("Location").agg({"Ratio LTV/CAC":"mean"}).sort_values(by="Ratio LTV/CAC",ascending=False)

,Ratio LTV/CAC
Location,
New York,32.785849
Kentucky,30.392803
South Dakota,30.023524
California,27.714002
Hawaii,27.288639
North Dakota,26.591688
Ohio,25.319319
Rhode Island,25.084850
Maryland,24.957436


### 5.12 Entre los clientes con `Previous Purchases` > X, ¿cuál es el gasto promedio y el LTV promedio?
Define un umbral X según tu contexto. Por ejemplo, X = 20:

In [17]:
df.groupby("Previous Purchases").agg({"Purchase Amount (USD)":"mean","LTV":"mean"})

,Purchase Amount (USD),LTV
Previous Purchases,,
1,58.469880,1108.349398
2,60.333333,1012.319444
3,58.560440,1137.516484
4,61.274725,1169.351648
5,64.252874,1201.436782
6,55.850575,1079.402299
7,61.200000,1078.292308
8,66.671642,1073.731343
9,63.784615,1111.584615


## 6. Conclusiones

En este Notebook, hemos abordado un **EDA orientado a negocio** en el que:
1. **Identificamos** la base total de clientes y su edad promedio.
2. **Calculamos** el CAC y LTV promedio, y la ratio LTV/CAC global.
3. **Segregamos** por categorías, suscripción, uso de descuentos, métodos de pago, etc.
4. **Resolvimos** preguntas de negocio que ayudan a:
   - **Identificar** cuáles categorías o ubicaciones ofrecen mejor rentabilidad (ratio LTV/CAC).
   - **Entender** si la suscripción eleva el LTV.
   - **Explorar** la efectividad de promos y descuentos.
   - **Diferenciar** frecuencias de compra y ver su impacto en LTV.
   - **Focalizar** inversiones de marketing en los segmentos más prometedores.

Este tipo de **análisis** proporciona las bases para **estrategias** de retención, adquisición y segmentación de clientes. ¡Espero que te sea útil para **generar valor** en tu negocio!